# Bitacora de Manual Feature Selection

En este notebook explicamos como hemos hecho la seleccion de variables. No hay ejecuciones, vamos a mencionar unicamente porque fuimos eliminando cada variable.

Para comenzar hablamos del baseline, donde se ejecuto el modelo por defecto eliminando las variables de cadenas. Se contaba con 34 variables iniciales y se obtuvo un r2 de 0.8730 con un rmse de 1272.

Luego de agregar las variables obtenidas el promedio de r2 se fue a 0.9028 con un promedio RMSE de 1066.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

df = ln_transform.copy()

Vamos a usar el summary y la matriz de correlacion para ir descartando variables.

In [ ]:
def corr_matrix(df, annotated=True):
    corr_matrix = df.corr(method='pearson')
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=annotated, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Matriz de Correlación')
    plt.show()

In [ ]:
corr_matrix(df, annotated=False)

Vamos a empezar eliminando Central Lock que tiene alta colinealidad con Powered Windows y en los summary tiene t-values muy bajos con alto P>|t|. El modelo no sufre casi nada.

La siguiente es Met_Color que en los summary tiene t-values muy bajos con alto P>|t|. Ademas de poca correlacion con precio. El modelo mejora con este drop.

Con el mismo criterio se elimina los features: "Central_Lock", "Met_Color", "Airbag_2", "ABS", "Backseat_Divider", "Metallic_Rim", "Radio", "Diesel", "Airbag_1", "Sport_Model", "m_vvti", "m_16v".

Automatic tenia malos valores pero lo estabamos haciendo aguantar porque creaiamos que podia influir, luego de borrar todas las anteriores los t-values siguen siendo cercanos a -0.9 con P>|t| de 0.3. Decidimos sacarla finalmente porque tampoco tiene buena correlacion de person.

El modelo tiene en la validacion cruzada con k-fold un promedio de:
- r2 = 0.9065
- rmse = 1052
- mae = 785

Vamos a hacer de nuevo otra matriz sin las columnas ya eliminadas.

In [ ]:
df.drop(columns=["Central_Lock", "Met_Color", "Airbag_2", "ABS", "Backseat_Divider", "Metallic_Rim", "Radio", "Diesel", "Airbag_1", "Sport_Model", "m_vvti", "m_16v", "Automatic"], inplace=True)
corr_matrix(df, annotated=False)

"Gears", "m_sedan", "m_bns", "m_wagon" son otras features obvias de sacar segun el summary y la correlacion.

"Power_Steering" y "Mistlamps" parecen empezar a ser las ultimas obvias, con t-values de entre -1.5 y -2.5, y con P>|t| de 0.1 aproximadamente. Al sacarlas el modelo empeora 0.01 en r2 y 1 punto en RMSE, pero mejora en BIC con todos los folds. Mantenemos la decision de sacar las variables.

Sacando las variables anteriores "Tow_Bar" empeora sus t-values. La situacion es similar a las variables anteriores, mejora BIC pero empeora demasiado poco en los otros valores para ser relevante.

Doors se comporta de manera extrania. En dos folds tiene buenos valores decentes de P>|t| y en otros dos supera los 0,2. Los coeficientes sin embargo son generalmente bajos, y teniendo en cuenta que no hay mucha relacion con precio la sacamos. El modelo mejora los BIC en los folds y solo empeora 0,0002 en r2 promedio, mantenemos el cambio.

Misma situacion con m_matic4 y m_matic3. 

In [ ]:
df.drop(columns=["Gears", "m_sedan", "m_bns", "m_wagon", "Power_Steering", "Mistlamps", "Tow_Bar", "Five_Doors", "Trunk", "m_exec", "m_matic4", "m_matic3"], inplace=True)
corr_matrix(df, annotated=True)

En este punto todas las variables ya estan con t-values mayores a 2 y P>|t| menores a 0,05. Empezaremos a darle mas importancia a la correlacion con el precio a la hora de borrar features, aunque no dejaremos de tener en cuenta estos valores. 

"m_g6", "m_gtsi" y "m_sport" tienen correlaciones muy bajas y sus t-values cercanos a 2. Sus coeficientes tambien son bajos respecto al resto. Eliminamos.

"cc" no tiene mucha relacion con precio y tiene cierta colinealidad con weight. Empeora bastante mas de lo esperado, volvemos atras.

Probamos sacando BoardComputer, que esta muy relacionada con CD_Player y Mfg_Year. No empeora casi nada, seguimos con este.

Al ir eliminando variables los t-values de m_sol, m_luna y m_terra empeoraron bastante. Son eliminados en orden inverso al mencionado. Luego, lo mismo sucede con "m_comfort", aunque con esta ultima solo mejora el BIC.

In [ ]:
df.drop(columns=["m_g6", "m_gtsi", "m_sport", "Boardcomputer", "m_sol", "m_luna", "m_terra", "m_comfort"], inplace=True)
corr_matrix(df, annotated=True)

Airco tiene alta correlacion con precio pero su coeficiente es muy bajo. Ademas su relacion con peso y anio puede estar influenciando. Eliminamos y el modelo empeora levemente. Vamos a probar otra variable.

"BOVAG_Guarantee" tiene muy poca relacion con precio y coeficientes relativamente bajos con respecto al resto. Igual que el anterior.

Probamos "Powered_Windows". Peor que los anteriores.

Parecemos haber llegado a un punto de inflexion donde quitar variables empeora sistematicamente el modelo en todos sus valores.

Vamos a eliminar las columnas de igual modo. Porque no empeora demasiado.

Sacando "Mfr_Guarantee" el modelo mejora un poco hacia el final. Por lo que lo sacamos.

Seguimos probando con "m_batch_b" y "m_liftb" que tienen coeficientes bajos en comparacion al resto.

Lo mismo con "m_d4d".

A partir de aqui el modelo solo empeora, consideramos que no tiene sentido seguir eliminando variables. En la carpeta de notebooks del proyecto hay dos imagenes con los valores promedios de validacion cruzada (r2, mse, rmse y mae) para cada ejecucion en mlflow.